In [2]:
import torch
import sys
import json
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
import ast
import os
import matplotlib.pyplot as plt

from src.dataset import ECommerceDS
from src.metrics import compute_recall_at_k, compute_mrr
from src.metric_handler import MetricHandler

sys.path.append("src/")
from models.sr_gnn import SRGNN

In [3]:
product2token_fp = "data/product2token.json"
weight_fp = "results/srgnn_baseline/best_model.pth"
device = "mps"
test_ds_fp = "data/splits/test.jsonl"
output_dir = 'results/srgnn_baseline/test_results'
N = 10 # num bins

os.makedirs(output_dir, exist_ok=True)

In [4]:
with open(product2token_fp, mode="r") as f:
    product2token = json.load(f)

In [5]:
model = SRGNN(hidden_size=128, n_node=len(product2token), num_layers=2)
model.load_state_dict(torch.load(weight_fp))
model.to(device)
model.eval()
print("Loaded model weights")

/var/folders/1n/tpv7t5v15pd7j42g_46lx79w0000gn/T/ipykernel_5001/2709022576.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_fp))


Loaded model weights


In [6]:
test_ds = ECommerceDS(test_ds_fp, max_len=50, product2token=product2token, mask="last")
test_dl = DataLoader(test_ds, batch_size=256, shuffle=False)

In [14]:
metric_handler = MetricHandler("results/srgnn_baseline")

all_logits = list()
all_labels = list()
all_times = list()

with torch.no_grad():  # Disable gradient computation for evaluation
    for test_batch in tqdm(test_dl):
        x_test = test_batch["graph"].to(device)
        labels_test = test_batch["products"].to(device)
        cloze_mask_test = test_batch["cloze_mask"].to(device)
        alibi = test_batch["alibi"].to(device)
        attention_mask = test_batch["attention_mask"].to(device)

        logits_flat_test = model(x_test)
        labels_flat_test = labels_test[:, -1]
        cloze_mask_flat_test = cloze_mask_test[:, -1]

        # Select only the masked positions
        valid_indices_test = cloze_mask_flat_test == 1
        logits_masked_test = logits_flat_test[valid_indices_test]
        labels_masked_test = labels_flat_test[valid_indices_test]

        all_logits.append(logits_masked_test)
        all_labels.append(labels_masked_test)
        time_vars = list()
        time_avgs = list()
        for i in range(labels_masked_test.shape[0]):
            time_var = torch.var(-torch.diff(alibi[i][-1][attention_mask[i]==1]))
            time_avg = torch.mean(-torch.diff(alibi[i][-1][attention_mask[i]==1]))
            time_vars.append(time_var)
            time_avgs.append(time_avg)
        time_vars = torch.tensor(time_vars)
        all_time_vars.append(time_vars)
        all_time_avgs.append(time_avgs)

  0%|          | 0/700 [00:00<?, ?it/s]/var/folders/1n/tpv7t5v15pd7j42g_46lx79w0000gn/T/ipykernel_59132/712113607.py:19: UserWarning: MPS: nonzero op is supported natively starting from macOS 14.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1728928937407/work/aten/src/ATen/native/mps/operations/Indexing.mm:361.)
  logits_masked_val = logits_flat_val[valid_indices_val]
100%|██████████| 700/700 [02:59<00:00,  3.91it/s]


### Next steps
- performance vs time variation
- by category

In [15]:
test_ds[0]

{'graph': Data(x=[4, 1], edge_index=[2, 12], edge_weights=[12]),
 'masked_products': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 2, 4, 5,
         5, 1]),
 'products': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 2, 4, 5,
         5, 6]),
 'times': ['2020-01-05 03:49:57 UTC',
  '2020-01-21 06:20:35 UTC',
  '2020-01-21 06:24:57 UTC',
  '2020-01-23 04:00:00 UTC',
  '2020-01-28 08:08:00 UTC',
  '2020-01-31 09:46:24 UTC',
  '2020-02-03 04:27:15 UTC',
  '2020-02-05 09:12:57 UTC'],
 'attention_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1]),
 'cloze_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 